# Demo: Using `create_odds_ratios` in the vivainsights Python Package

This notebook demonstrates how to use the `create_odds_ratios` function from the **vivainsights** Python package. The function calculates odds ratios for ordinal metrics against a specified independent variable.

In this example, we will:
1. Load the demo data (`pq_data`) from the package.
2. Use `identify_usage_segments` to create a metric for the independent variable (`UsageSegments_12w`).
3. Calculate odds ratios for ordinal metrics against the independent variable using `create_odds_ratios`.
4. Visualize the results.


In [ ]:
# Import necessary libraries
import vivainsights as vi
import pandas as pd
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

## Step 1: Load the demo data

We will load the sample Person Query dataset (`pq_data`) provided by the **vivainsights** package.

In [ ]:
# Load the demo data
pq_data = vi.load_pq_data()

# Display the first few rows of the dataset
pq_data.head()

## Step 2: Create the independent variable using `identify_usage_segments`

We will use the `identify_usage_segments` function to classify users into usage segments based on their Copilot actions. The independent variable (`UsageSegments_12w`) will be created using the `metric_str` method, aggregating all columns that start with `Copilot_actions_taken_in_`.

In [ ]:
# Identify usage segments
usage_segments_data = vi.identify_usage_segments(
    data=pq_data,
    metric_str=[
        "Copilot_actions_taken_in_Teams",
        "Copilot_actions_taken_in_Outlook",
        "Copilot_actions_taken_in_Excel",
        "Copilot_actions_taken_in_Word",
        "Copilot_actions_taken_in_Powerpoint"
    ],
    version="12w",
    return_type="data"
)

# Display the first few rows of the updated dataset
usage_segments_data.head()

## Step 3: Calculate odds ratios for ordinal metrics

We will calculate odds ratios for the following ordinal metrics:
- `eSat`
- `Initiative`
- `Manager_Recommend`
- `Resources`
- `Speak_My_Mind`
- `Wellbeing`
- `Work_Life_Balance`
- `Workload`

The independent variable will be `UsageSegments_12w`.

In [ ]:
# Define the ordinal metrics
ordinal_metrics = [
    "eSat", "Initiative", "Manager_Recommend", "Resources",
    "Speak_My_Mind", "Wellbeing", "Work_Life_Balance", "Workload"
]

# Calculate odds ratios
odds_ratios_table = vi.create_odds_ratios(
    data=usage_segments_data,
    ord_metrics=ordinal_metrics,
    metric="UsageSegments_12w",
    ord_item_options=5,  # Assuming a 5-point scale for ordinal metrics
    return_type="table"
)

# Display the odds ratios table
odds_ratios_table

## Step 4: Visualize the odds ratios

We will create a bar plot to visualize the odds ratios for the ordinal metrics.

In [ ]:
# Visualize odds ratios
odds_ratios_plot = vi.create_odds_ratios(
    data=usage_segments_data,
    ord_metrics=ordinal_metrics,
    metric="UsageSegments_12w",
    ord_item_options=5,
    return_type="plot"
)

# Display the plot
odds_ratios_plot.show()

## Summary

In this notebook, we demonstrated how to:
1. Load the demo data (`pq_data`).
2. Create an independent variable (`UsageSegments_12w`) using `identify_usage_segments`.
3. Calculate odds ratios for ordinal metrics using `create_odds_ratios`.
4. Visualize the results.

The `create_odds_ratios` function provides a powerful way to analyze the relationship between ordinal metrics and an independent variable.